In [128]:
%pwd

'/Users/whenessel/Development/PycharmProjects/appsurify-testbrain-core/src'

In [129]:

import os
import sys
from asgiref.sync import sync_to_async
import pandas as pd

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "system.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "false"

import django

django.setup()

from django.conf import settings
from django.db.models import Subquery, OuterRef, Exists, Q, Value, F
from django.db.models.functions import MD5, Coalesce
from applications.vcs.models import Commit, File, FileChange

In [130]:
commit_qs = Commit.objects.filter(project_id=469, sha__in=["421b77eb6e25e4022875b85896a9937f319605e9", "7dc174e02612fd51428ac30bde0d0d3079f7e890", "503451ff079656b464ad5a8d1fd9d5f476a16c2d"])\
    .order_by("-timestamp")\
    .values("sha", "files__full_filename")\
    .annotate(file_id=Coalesce(MD5("files__full_filename"), MD5(Value(""))))\
    .values("sha", "file_id")

commit_qs = commit_qs.order_by("?")[:100]

file_qs = File.objects.filter(project_id=469, children__isnull=True).only("full_filename")\
    .annotate(file_id=MD5("full_filename"), file_name=F("full_filename"))\
    .values("file_id", "file_name")


In [131]:
df_commits = pd.DataFrame(commit_qs)
df_commits["file_id"].fillna(0, inplace=True)
df_commits["file_id"] = df_commits["file_id"].astype("str")
print(df_commits.shape)
df_commits.info(memory_usage="deep")

(100, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   sha      100 non-null    object
 1   file_id  100 non-null    object
dtypes: object(2)
memory usage: 18.3 KB


In [132]:
df_commits

,sha,file_id
0,421b77eb6e25e4022875b85896a9937f319605e9,10a5f1004ff2dd28b22f7f9afa36f383
1,421b77eb6e25e4022875b85896a9937f319605e9,89332d1ba456ba8c882048f633247f70
2,421b77eb6e25e4022875b85896a9937f319605e9,1684c91333183206aea75054a6bfafdf
3,421b77eb6e25e4022875b85896a9937f319605e9,af7a82cf4a449c05a69e5594e4a4eeb6
4,503451ff079656b464ad5a8d1fd9d5f476a16c2d,6faf32fc346a5d3de680c7c3895aba3c
...,...,...
95,421b77eb6e25e4022875b85896a9937f319605e9,a8726e0525f5ab7da46659c75a267a10
96,421b77eb6e25e4022875b85896a9937f319605e9,b53e1c2ac61f5ac6c577115d4c80585b
97,421b77eb6e25e4022875b85896a9937f319605e9,581e9982580905b20099384a9acf69bb
98,421b77eb6e25e4022875b85896a9937f319605e9,bcaebf5a102b5b7b5c88e0987c73413d


In [133]:
df_files = pd.DataFrame(file_qs)
df_files["file_id"] = df_files["file_id"].astype("str")
print(df_files.shape)
df_files.info(memory_usage="deep")

(5552, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5552 entries, 0 to 5551
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_id    5552 non-null   object
 1   file_name  5552 non-null   object
dtypes: object(2)
memory usage: 1.1 MB


In [134]:
df_files.head()

,file_id,file_name
0,45f215578976c5f77317d3b5788561ce,src/assets/sass/list-lander/desktop/_table.scss
1,7cfbb3ad01ece9d907bf5234d1e85ba7,src/assets/sass/list-lander/desktop/_more-list...
2,c0eb8cbab7743c73ae76525bdad64c40,src/assets/sass/list-lander/desktop/lists/clou...
3,e0d88f3bd314687d9ec0d5ba5012b516,src/assets/sass/list-lander/desktop/lists/cana...
4,71871959fe4a466bfe5b4038fb517c94,src/assets/sass/list-lander/desktop/lists/shar...


In [135]:
df_sum = df_commits.merge(df_files, how="cross").sort_values(by=["sha"])
print(df_sum.shape)
df_sum.info(memory_usage="deep")

(555200, 4)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 555200 entries, 0 to 338669
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   sha        555200 non-null  object
 1   file_id_x  555200 non-null  object
 2   file_id_y  555200 non-null  object
 3   file_name  555200 non-null  object
dtypes: object(4)
memory usage: 209.1 MB


In [140]:
df_sum["state"] = df_sum.apply(lambda x: 1 if x.loc["file_id_x"] == x.loc["file_id_y"] else 0, axis=1)
df_sum["state"] = df_sum["state"].astype("int8")
print(df_sum.shape)
df_sum.info(memory_usage="deep")

(555200, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 555200 entries, 0 to 338669
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   sha        555200 non-null  object
 1   file_id_x  555200 non-null  object
 2   file_id_y  555200 non-null  object
 3   file_name  555200 non-null  object
 4   state      555200 non-null  int8  
dtypes: int8(1), object(4)
memory usage: 209.6 MB


In [141]:
df_sum

,sha,file_id_x,file_id_y,file_name,state
0,421b77eb6e25e4022875b85896a9937f319605e9,10a5f1004ff2dd28b22f7f9afa36f383,45f215578976c5f77317d3b5788561ce,src/assets/sass/list-lander/desktop/_table.scss,0
351634,421b77eb6e25e4022875b85896a9937f319605e9,32607347f8126e6534ebc7ebaec4853d,84dff5560ffd49ac8c8b75ffb0801d4c,src/views/chansec/cards/four_articles_across_c...,0
351633,421b77eb6e25e4022875b85896a9937f319605e9,32607347f8126e6534ebc7ebaec4853d,b7a4fe6d376bc063522956089eb42406,src/views/chansec/cards/feature-and-stream-pre...,0
351632,421b77eb6e25e4022875b85896a9937f319605e9,32607347f8126e6534ebc7ebaec4853d,c59ddb7836960e54adbadd3441b5bd78,src/views/chansec/cards/three-article-feature-...,0
351631,421b77eb6e25e4022875b85896a9937f319605e9,32607347f8126e6534ebc7ebaec4853d,576e34d2f2fe6f3df9e84a77ee724e85,src/views/chansec/cards/special-feature-card.pug,0
...,...,...,...,...,...
334965,7dc174e02612fd51428ac30bde0d0d3079f7e890,d41d8cd98f00b204e9800998ecf8427e,5f02081a0e4eb6dafe4255f50049dc93,src/views/shared/ads-wrappers.pug,0
334964,7dc174e02612fd51428ac30bde0d0d3079f7e890,d41d8cd98f00b204e9800998ecf8427e,3c8f4fb9d32f1d36712f20387005d3bb,src/views/shared/social-media.pug,0
334963,7dc174e02612fd51428ac30bde0d0d3079f7e890,d41d8cd98f00b204e9800998ecf8427e,4c337a7d85c88510676d0df56718a3ca,src/views/shared/share-buttons.pug,0
334971,7dc174e02612fd51428ac30bde0d0d3079f7e890,d41d8cd98f00b204e9800998ecf8427e,c323d43d18fd1c9588b6ba0d303e9686,src/views/shared/user-management-modal.pug,0


In [142]:
df_sum[["sha", "file_name", "state"]].to_csv("./CMerged.csv", sep="\t", index=False, header=True)

In [139]:
# df_sum[["sha", "file_name", "changed"]].to_csv("./C421b77eb6e25e4022875b85896a9937f319605e9.csv", sep="\t", index=False, header=True)